In [54]:
#!/usr/bin/env python3
'''
Based on the work done in 
On-Manifold Preintegration for Real-Time Visual-Inertial Odometry,
VINS-Mono: A Robust and Versatile Monocular Visual-Inertial State Estimator,
and Vision-Based Control of Multirotor Systems (yet to be published)
'''

import numpy as np
from scipy.linalg import expm, logm
import matplotlib.pyplot as plt

def skew(v):
    return np.array([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])

In [55]:
class IMU:
    def __init__(self):
        self.ba = np.zeros(3) #Leave as zeros for now
        self.bw = np.zeros(3)
        self.covar_a = np.diag([0.00561, 0.00561, 0.00561]) #look up good value for this
        self.covar_w = np.diag([0.00025, 0.00025, 0.00025]) #look up good values for this
    
    def getAccel(self, a_nom, w_nom):
        noise_a = np.random.multivariate_normal(np.zeros(3), self.covar_a)
        noise_w = np.random.multivariate_normal(np.zeros(3), self.covar_w)
        a = a_nom + self.ba + noise_a
        w = w_nom + self.bw + noise_w
        
        return a, w

In [56]:
class State:
    def __init__(self, p=np.zeros(3), v=np.zeros(3), R=np.eye(3), w=np.zeros(3)):
        self.p_i = p
        self.v_i = v
        self.Ri_from_b = R
        self.w_b = w
    
    def __mul__(self, dt):
        self.p_i = self.p_i * dt
        self.v_i = self.v_i * dt
        self.Ri_from_b = self.Ri_from_b * dt
        self.w_b = self.w_b * dt #Note that this isn't alpha. It is still omega
        
        return self
    
    def __rmul__(self, val):
        return self * val
    
    def __add__(self, dx):
        self.p_i = self.p_i + dx.p_i
        self.v_i = self.v_i + dx.v_i
        self.Ri_from_b = self.Ri_from_b @ expm(skew(dx.w_b))
        
        return self
    
    def normalize(self): #Graham Schmidt orthonormalization or rotation matrix
        x = self.Ri_from_b[:,0] / np.linalg.norm(self.Ri_from_b[:,0])
        y = skew(self.Ri_from_b[:,2]) @ x
        y = y / np.linalg.norm(y)
        z = skew(x) @ y
        z = z/np.linalg.norm(z)
        self.Ri_from_b = np.array([[*x], [*y], [*z]]).T

class Input:
    def __init__(self):
        self.a_b = np.zeros(3)
        self.w_b = np.zeros(3)
    
    def update(self, a, w):
        self.a_b = a
        self.w_b = w

In [57]:
class Multirotor:
    def __init__(self, m, J):
        self.m = m
        self.J = J
        self.g = 9.81
        
        # Define states for the multirotor
        self.state = State()
    
    def dynamics(self, state, u):
        v_i = state.v_i
        Ri_from_b = state.Ri_from_b
        a_b = u.a_b
        w_b = u.w_b
        e3 = np.array([0, 0, 1])
        
        state_dot = State()
        state_dot.p_i = v_i
        state_dot.v_i = self.g * e3 + Ri_from_b @ a_b
        state_dot.Ri_from_b = Ri_from_b @ skew(w_b)
        state_dot.w_b = w_b #Not the derivative but this will be used in the integration
        
        return state_dot
    
    def update(self, u, dt):
        k1 = self.dynamics(self.state, u)
        k2 = self.dynamics(self.state + k1 * (dt/2), u)
        k3 = self.dynamics(self.state + k2 * (dt/2), u)
        k4 = self.dynamics(self.state + k3 * dt, u)
        
        self.state = self.state + (dt/6) * (k1 + 2 * k2 + 2*k3 + k4)
        return self.state


In [58]:
mass = 2.856
J = np.diag([.07, .08, .12])

quad1 = Multirotor(mass, J)
# #Test to see if dynamics work
# for i in range(100):
#     a = np.array([1, 0, 1])
#     w = np.array([.1, 0.0, .1])
#     dt = .01
#     u = Input()
#     u.update(a, w)
#     temp = quad1.update(u, dt)
#     if not np.linalg.det(temp.Ri_from_b) == 1.0:
#         quad1.state.normalize()
# print(temp.p_i)
# print(temp.v_i)
# print(temp.Ri_from_b)
# print(np.linalg.det(temp.Ri_from_b))